In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import uproot
import awkward as ak


# Getting DataFrame, no osc samples

In [ ]:
loc = "BNBNuMI_TOsc_input_numifluxpatched/hist_rootfiles/"

beams = [
    "BNB", 
    "NuMI",
]

runs = [
    "run1",
    "run2",
    "run3",
]

horns = [
    "FHC",
    "RHC",
]

sels = [
    ("nueCC", "_FC"),
    ("nueCC", "_PC"),
    ("numuCC", "_FC"),
    ("numuCC", "_PC"),
    ("CCpi0", "_FC"),
    ("CCpi0", "_PC"),
    ("NCpi0", ""),
]

filetypes = [
    ("intrinsic", "intnue"), 
    ("nu_overlay", "overlaynumu"),
    ("dirt", "???"),
    ("ext", "???"),
    ("data", "???"),
]

for beam in beams:
    for filetype_pair in filetypes:
        for run in runs:
            for horn in horns:

                if beam == "NuMI":
                    beam = "NuMI_"+horn

                filename = f"roofile_obj_{beam}_{run}_{filetype_pair[0]}.root"
                print(loc+filename)
                f = uproot.open(loc+filename)

                for sel in sels:

                    try:
                        key = f"tree_{sel[0]}_from_{filetype_pair[1]}{sel[1]}"
                        print(f"    key: {key}, {f[key].keys()}")
                    except KeyError:
                        #print(f"    KeyError: {key}")
                        pass

